#### 📝 Step 3.1: Convert shape_ann/ and texture_ann/ to YOLO Format



In [1]:
import os
from tqdm import tqdm  # For progress bar

# 🔹 Define Paths
dataset_path = r"C:\Users\raiha\OneDrive\Desktop\cv_dataset\deepfashion2"
texture_ann_path = os.path.join(dataset_path, "texture_ann")
shape_ann_path = os.path.join(dataset_path, "shape_ann")

# YOLO Output Folder
labels_yolo_path = os.path.join(dataset_path, "labels_yolo")
os.makedirs(labels_yolo_path, exist_ok=True)

# 🔹 Convert Bounding Box to YOLO Format
def convert_bbox_to_yolo(x, y, w, h, img_w, img_h):
    """Convert absolute bbox coordinates to YOLO format (normalized)."""
    x_center = (x + w / 2) / img_w
    y_center = (y + h / 2) / img_h
    w_norm = w / img_w
    h_norm = h / img_h
    return x_center, y_center, w_norm, h_norm

# 🔹 Function to Process Annotations
def process_annotations(ann_path, label_folder, img_w, img_h, prefix=""):
    os.makedirs(label_folder, exist_ok=True)
    for ann_file in tqdm(os.listdir(ann_path), desc=f"Processing {prefix}annotations"):
        if ann_file.endswith(".txt"):
            ann_file_path = os.path.join(ann_path, ann_file)
            label_file_path = os.path.join(label_folder, f"{prefix}{ann_file}")

            try:
                with open(ann_file_path, "r") as f, open(label_file_path, "w") as yolo_f:
                    for line in f.readlines():
                        data = line.strip().split()
                        if len(data) < 5:
                            continue  # Skip invalid lines

                        # 🔹 Check if first element is an integer (class ID)
                        try:
                            class_id = int(data[0])  # Class ID
                        except ValueError:
                            continue  # Skip if not an integer (image filename detected)

                        # Bounding box values
                        x, y, w, h = map(float, data[1:5])

                        # Convert to YOLO format
                        x_center, y_center, w_norm, h_norm = convert_bbox_to_yolo(
                            x, y, w, h, img_w, img_h
                        )

                        # Save YOLO annotation
                        yolo_f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")
            except Exception as e:
                print(f"Error processing {ann_file}: {e}")

# 🔹 Process Texture Annotations
for subset in ["train", "val", "test"]:
    ann_subset_path = os.path.join(texture_ann_path, subset)
    labels_subset_path = os.path.join(labels_yolo_path, f"texture_{subset}")
    process_annotations(ann_subset_path, labels_subset_path, img_w=256, img_h=256)

# 🔹 Process Shape Annotations
process_annotations(shape_ann_path, os.path.join(labels_yolo_path, "shape_"), img_w=256, img_h=256)

print("✅ Annotations converted! YOLO labels saved in 'labels_yolo/' folder.")


Processing annotations: 100%|██████████| 3/3 [00:00<00:00, 44.84it/s]

✅ Annotations converted! YOLO labels saved in 'labels_yolo/' folder.


In [1]:
from ultralytics import YOLO

# Load a pre-trained YOLOv8 model
model = YOLO("yolov8m.pt")  # You can also use 'yolov8m.pt' or 'yolov8l.pt'

# Train the model on DeepFashion2 dataset
model.train(data=r"C:\Users\raiha\OneDrive\Desktop\cv_dataset\deepfashion2\deepfashion2.yaml", epochs=3, imgsz=320)


100%|██████████| 49.7M/49.7M [04:24<00:00, 197kB/s] 


New https://pypi.org/project/ultralytics/8.3.101 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.97  Python-3.12.9 torch-2.2.1+cpu CPU (AMD Ryzen 5 5500U with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=C:\Users\raiha\OneDrive\Desktop\cv_dataset\deepfashion2\deepfashion2.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=N

train: Scanning C:\Users\raiha\OneDrive\Desktop\cv_dataset\deepfashion2\train_images.cache... 0 images, 10335 backgrounds, 0 corrupt: 100%|██████████| 10335/10335 [00:00<?, ?it/s]

WARNING  No labels found in C:\Users\raiha\OneDrive\Desktop\cv_dataset\deepfashion2\train_images.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.



val: Scanning C:\Users\raiha\OneDrive\Desktop\cv_dataset\deepfashion2\test_images.cache... 0 images, 1149 backgrounds, 0 corrupt: 100%|██████████| 1149/1149 [00:00<?, ?it/s]

WARNING  No labels found in C:\Users\raiha\OneDrive\Desktop\cv_dataset\deepfashion2\test_images.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


Plotting labels to c:\Users\raiha\OneDrive\Desktop\AR-Shopping-Assistant\runs\detect\train10\labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000455, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 320 train, 320 val
Using 0 dataloader workers
Logging results to c:\Users\raiha\OneDrive\Desktop\AR-Shopping-Assistant\runs\detect\train10
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/3         0G          0      6.341          0          0        320: 100%|██████████| 646/646 [2:21:20<00:00, 13.13s/it]   
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [01:58<00:00,  3.29s/it]


RuntimeError: torch.cat(): expected a non-empty list of Tensors

In [ ]:
# Validate the model
model.val(data=r"C:\Users\raiha\OneDrive\Desktop\cv_dataset\deepfashion2\deepfashion2.yaml")


In [ ]:
# Run inference on test images
results = model("C:/Users/raiha/OneDrive/Desktop/cv_dataset/deepfashion2/test_images/")
